In [ ]:
# ArrayList and Stack code to use

class Stack():
    def __init__(self):
        self.inList = None
        self.size = 0
        
    def push(self, v):
        self.size += 1
        self.inList = (v,self.inList)
        
    def pop(self):
        if self.size == 0: assert(0), "Popping from an empty Stack!"
        self.size -= 1
        (v,ls) = self.inList
        self.inList = ls
        return v
    
    def __str__(self):
        s = "["
        ls = self.inList
        for _ in range(self.size):
            (v,ls) = ls
            s += str(v)
            if ls!=None: s += ", "
        return s+"]"
        
class ArrayList:
    def __init__(self):
        self.inArray = [0 for i in range(10)]
        self.count = 0
        
    def get(self, i):
        return self.inArray[i]

    def set(self, i, e):
        self.inArray[i] = e

    def length(self):
        return self.count

    def append(self, e):
        self.inArray[self.count] = e
        self.count += 1
        if len(self.inArray) == self.count:
            self._resizeUp()

    def insert(self, i, e):
        for j in range(self.count,i,-1):
            self.inArray[j] = self.inArray[j-1]
        self.inArray[i] = e
        self.count += 1
        if len(self.inArray) == self.count:
            self._resizeUp()
    
    def remove(self, i):
        self.count -= 1
        val = self.inArray[i]
        for j in range(i,self.count):
            self.inArray[j] = self.inArray[j+1]
        return val

    def _resizeUp(self):
        newArray = [0 for i in range(2*len(self.inArray))]
        for j in range(len(self.inArray)):
            newArray[j] = self.inArray[j]
        self.inArray = newArray
        
    def toArray(self):
        return self.inArray[:self.count]

    def __str__(self):
        if self.count == 0: return "[]"
        s = "["
        for i in range(self.count-1): s += str(self.inArray[i])+", "
        return s+str(self.inArray[self.count-1])+"]" 
    
class ArrayListWithUndo(ArrayList):
    def __init__(self):
        super().__init__()
        self.undos = Stack()


    def set(self, i, v):
        self.undos.push(("set", i, self.inArray[i]))
        self.inArray[i] = v


    def append(self, v):
        super().append(v)
        self.undos.push(("rem", self.length()-1, None))
        

    def insert(self, i, v):
        super().insert(i, v)
        self.undos.push(("rem", i, None))
        

    def remove(self, i):
        self.undos.push(("ins", i, self.inArray[i]))
        super().remove(i)

    
    def undo(self):
        if self.undos.size == 0:
            return
        
        instruction, idx, val = self.undos.pop()

        if instruction == "set":
            self.inArray[idx] = val
        elif instruction == "rem":
            super().remove(idx)
        else:
            super().insert(idx, val)
        
            
    def __str__(self):
        # already implemented
        return str(self.toArray())+"\n-> "+str(self.undos)

class NetworkWithUndo:
    def __init__(self, N):
        self.inArray = ArrayListWithUndo()
        for _ in range(N): self.inArray.append(-1)
        self.undos = Stack()
        self.undos.push(N)
        
    def getSize(self):
        return self.inArray.length()
        
    def add(self): 
        self.inArray.append(-1)
        self.undos.push(1)

        return
    
    def root(self, i):
        indexToCheck = i
        visitedNodes = Stack()
        while self.inArray.get(indexToCheck) >= 0:
            visitedNodes.push(indexToCheck)
            indexToCheck = self.inArray.get(indexToCheck)

        rootIdx = indexToCheck
        self.undos.push(visitedNodes.size)

        while visitedNodes.size > 0: 
            currentIndex = visitedNodes.pop()
            self.inArray.set(currentIndex, rootIdx)       
        
        return rootIdx    
    
    def merge(self, i, j):
        iLength = self.inArray.get(i)
        jLength = self.inArray.get(j)
        newLength = iLength + jLength

        if iLength < jLength:
            self.inArray.set(i, newLength)
            self.inArray.set(j, i)
        else:
            self.inArray.set(j, newLength)
            self.inArray.set(i, j)

        self.undos.push(2)

        return
    
    def undo(self):
        if self.undos.size > 0:
            numberOfOperations = self.undos.pop()

            for _ in range(numberOfOperations):
                self.inArray.undo()
        
        return
    
    def toArray(self):
        return self.inArray.toArray()
            
    def __str__(self):
        return str(self.toArray())+"\n-> "+str(self.undos)
    
class Gadget:
    def __init__(self):
        self.inNetwork = NetworkWithUndo(0)
        self.subsize = 0
        self.nameMap = {}
        self.undos = Stack()
        self.helper = {} # use name to track the size of undo before adding node
        
    def getSize(self):
        # already implemented
        return self.inNetwork.getSize()
        
    def isIn(self, name):
        # already implemented
        return name in self.nameMap
        
    def add(self, name):
        if self.isIn(name): # return if name is in Gadget
            self.undos.push(("oth", 0, None))
            return 
        else:
            self.nameMap[name] = self.getSize() #map the unique integer N-1 to name 
            self.helper[name] = self.undos.size # get size of undos before carrying out any operation
            self.subsize += 1 #new node added, so increment the subsize by 1
            self.inNetwork.add() #add a new node to the self.inNetwork
            self.undos.push(("rem", 1, name)) #push the undo instruction triplet of add

    
    def connect(self, name1, name2):
        node1 = self.nameMap[name1]
        node2 = self.nameMap[name2]

        rootNode1 = self.inNetwork.root(node1)
        rootNode2 = self.inNetwork.root(node2)
        if rootNode1 == rootNode2: # checking to see if connected i.e. in same cluster
            self.undos.push(("oth", 2, None))
            return True
        else: 
            self.subsize -= 1 #2 nodes connected combining 2 clusters into one
            self.inNetwork.merge(rootNode1, rootNode2) # merging the nodes of the corresponding network
            self.undos.push(("brk", 3, None)) #push the undo instruction triplet of break
            return False
        
        
    def clean(self, name):
        if not self.isIn(name):
            return # name not in self.nameMap
        
        numUndos = self.undos.size - self.helper[name] # find out number of times to call undo from current stack size to old stack size
        self.undo(numUndos)

    def subnets(self):
        # return an arr containing all subnets(clusters) in the network
        # arr should contain arrays of strings(keys in self.nameMap)

        arr = []
        clusters = {}
        for string in self.nameMap.keys(): #by this for loop, we are trying to group all the strings that have the same root node(i.e. are in the same cluster)
            index = self.nameMap[string] #find the network index string maps to
            rootIdx = self.inNetwork.root(index) #find the root node of it

            if rootIdx not in clusters: #initialize with empty array if the key rootIdx is not in clusters
                clusters[rootIdx] = []

            clusters[rootIdx].append(string) #append the string to the array in clusters[rootIdx]
        
        for arrayOfStrings in clusters.values(): # append all the arrays of strings(each array represents a cluster) to arr
            arr.append(arrayOfStrings)
        
        self.undos.push(("oth", len(self.nameMap), None))

        return arr
        
    def undo(self, n):
        while n != 0 and self.getSize() != 0: # if we are led to an empty Gadget(subsize = 0), stop
            op, numberOfSteps, s = self.undos.pop()

            if op == "rem":
                self.nameMap.pop(s) #remove string s from the nameMap
                self.helper.pop(s) # remove string s from helper
                self.subsize -= 1 #reduce subsize by 1
            elif op == "brk":
                self.subsize += 1 #increase the subsize by 1

            for _ in range(numberOfSteps): #perform numberOfSteps undo steps on self.inNetwork
                self.inNetwork.undo()

            n -= 1

        return
        
    def toArray(self):
        A = self.inNetwork.toArray()
        for s in self.nameMap:
            i = self.nameMap[s]
            A[i] = (s,A[i])
        return A
        
    def __str__(self):
        return str(self.toArray())+"\n-> "+str(self.nameMap)+"\n-> "+str(self.undos)